In [339]:
import time
import requests
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import numpy as np

In [42]:
def get_content(url):
    page = requests.get(url)
    return BeautifulSoup(page.content, 'html.parser')
    

In [43]:
def get_text(soup):
    for script in soup(["script", "style"]):
            script.extract()
            
    text = soup.get_text()
    return re.sub(r"\s+", " ", text)

In [78]:
def get_links(soup):
    links = []
    links_clear = []

    for a in soup.find_all('a', href=True):
        links.append(a['href'])    
        
    for link in links:
        if "http" in link: 
            if link.startswith("https://www.sdglab.am"):
                links_clear.append(link)
            
    return list(set(links_clear))
    

In [197]:
def create_word_list(scrapped_website):
    word_list = []
    for x in scrapped_website:
        for li in x.split(" "):
            if re.match(r'[\u0561-\u0587\u0531-\u0556]', li):
                word_list.append(li)
            
    clear_word_list = []
    for word in word_list:
        for ch in [' ', ',', '.', '․', '?', '!', '։', ':', '-', '«', '»', '՛', '՝', '`', '(', ')', '/']:
            if ch in word:
                word = word.replace(ch, ' ')
        clear_word_list.extend(word.split(" "))

    clear_word_list = [x for x in clear_word_list if len(x) > 0]
    for cwl in range(len(clear_word_list)):
        clear_word_list[cwl] = clear_word_list[cwl].lower()
    clear_word_list = set(clear_word_list)
    clear_word_list = sorted(clear_word_list)
    return clear_word_list[1:]
    

In [309]:
def save_to_file(word_list, file_name):
    with open(file_name, 'w') as file:
        for wl in word_list:
            file.write(wl)
            file.write('\n')

In [357]:
def read_file(file_name):
    with open(file_name) as f:
        return [line.rstrip() for line in f]

In [354]:
def divide_chunks(li, n):
    for i in range(0, len(li), n): 
        yield li[i:i + n]

In [83]:
url='https://www.sdglab.am/%D5%B6%D5%B8%D6%80%D5%B8%D6%82%D5%A9%D5%B5%D5%B8%D6%82%D5%B6%D5%B6%D5%A5%D6%80'
links_buffer = []

def scrap_ormian_site(url):

    soup = get_content(url)
    text = [get_text(soup)]
    links = get_links(soup)
    links = [x for x in links if x not in links_buffer]
    links_buffer.extend(links)
    
    for link in links:
        soup_tmp = get_content(link)
        text.append(get_text(soup_tmp))
        links_tmp = get_links(soup_tmp)
        if len(links_tmp) != 0:
            text.extend(scrap_ormian_site(link))

    return text

scrapped_website = scrap_ormian_site(url)

In [311]:
word_list = create_word_list(scrapped_website)
save_to_file(word_list, 'armenian_corpus.txt')

In [379]:
lines = read_file('armenian_corpus.txt')
    
split = []
for dc in divide_chunks(lines, 400):
    split.append(dc)

In [378]:
options = Options()
options.add_argument("--headless")
driver = webdriver.Firefox(options=options)

with open('armenian_english_corpus.txt', 'a') as file:
    for line in split[6]:
        glosbe = 'https://glosbe.com/hy/en/'+line
        driver.get(glosbe)
        html = driver.page_source
        soup = BeautifulSoup(html)
        div = soup.body.find('h3', attrs={'class': 'translation'})

        if div == None:
            text = "-"
        else:
            text = div.get_text()
            text = re.sub(r"\s+", " ", text)
            text = text.replace(" ", "")
        file.write(f'{line}\t{text}\n')